In [38]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import re
import xgboost

from pandas.api.types import CategoricalDtype

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from statsmodels.graphics.tsaplots import plot_pacf
from xgboost import XGBRegressor
import lightgbm
from sklearn.model_selection import GridSearchCV
import joblib
import logging

In [39]:

all_csv = pd.read_csv('cleaned.csv.gz',
                 dtype = {
                     'store_nbr' : 'category',
                     'family' : 'category',
                     'sales': 'float',
                     'city': 'category',
                     'state': 'category',
                     'type': 'category',
                     'holiday_type': 'category',
                     'holiday_transferred': 'category'
                 },
                  parse_dates=['date'])
all_csv['date'] = pd.to_datetime(all_csv['date']).dt.to_period('D')

In [40]:
all = all_csv.copy()  # we can start experimenting from here without reloading the csv file

In [42]:
# this is for experimentation

filter_by_stores = None  # note: please use string here (unlike Mine.ipynb)
filter_by_family = None
filter_by_dates = None

#filter_by_stores = ['1', '2']  # note: please use string here (unlike Mine.ipynb)
#filter_by_family = ['DAIRY', 'PRODUCE']
#filter_by_family = ['']
#filter_by_dates = '2014-06-05'


In [43]:
if filter_by_dates == None:
    train_start_date = '2013-01-01'
else:
    train_start_date = filter_by_dates
train_end_date = '2017-08-15'
test_start_date = '2017-08-16'
test_end_date = '2017-08-31'

In [44]:

if filter_by_family != None:
    all = all[all['family'].isin(filter_by_family)]
if filter_by_stores != None:
    all = all[all['store_nbr'].isin(filter_by_stores)]
if filter_by_dates != None:
    all = all[all['date'] >= filter_by_dates]

In [45]:
all.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3036528 entries, 0 to 3036527
Data columns (total 38 columns):
 #   Column               Dtype    
---  ------               -----    
 0   date                 period[D]
 1   store_nbr            category 
 2   family               category 
 3   sales                float64  
 4   onpromotion          int64    
 5   sales_lag_01         float64  
 6   sales_lag_02         float64  
 7   sales_lag_03         float64  
 8   sales_lag_04         float64  
 9   sales_lag_05         float64  
 10  sales_lag_06         float64  
 11  sales_lag_07         float64  
 12  sales_lag_08         float64  
 13  sales_lag_09         float64  
 14  sales_lag_10         float64  
 15  sales_lag_11         float64  
 16  sales_lag_12         float64  
 17  sales_lag_13         float64  
 18  sales_lag_14         float64  
 19  sales_lag_15         float64  
 20  sales_lag_16         float64  
 21  sales_lag_17         float64  
 22  sales_lag_18      

In [46]:
def one_hot_encode(df):
    return pd.get_dummies(data=df, columns=['store_nbr', 'family', 'city', 'state', 'type',
                                     'cluster', 'holiday_type', 'holiday_transferred', 'weekday']) 

In [48]:
all_ohe = one_hot_encode(all)
all_ohe = all_ohe.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))  # remove bad char in column names

X = all_ohe[all_ohe['date'] <= train_end_date]
X = X.drop(['sales'], axis=1)
y = all_ohe[['date', 'sales']][all_ohe['date'] <= train_end_date]
y.set_index('date', inplace=True)

X_test = all_ohe[all_ohe['date'] >= test_start_date]
X_test = X_test.drop(['sales'], axis=1)

X.drop('date', axis=1, inplace=True)
X_test.drop('date', axis=1, inplace=True)
y.set_index(X.index, inplace=True)

# from sklearn.decomposition import PCA
# pca = PCA(n_components=180)
# principalComponents = pca.fit_transform(X)
# principalDf = pd.DataFrame(data = principalComponents)

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=1)

Random Forest Regressor

In [34]:
from sklearn.ensemble import RandomForestRegressor

model_rf = RandomForestRegressor(n_estimators = 225, n_jobs=-1, random_state=5)
model_rf.fit(X_train, y_train)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.



RandomForestRegressor(n_estimators=225, n_jobs=-1, random_state=5)

Random Forest Evaluation

In [52]:
y_pred_train = model_rf.predict(X_train)
y_pred_train[y_pred_train < 0] = 0
y_pred_val = model_rf.predict(X_val)
y_pred_val[y_pred_val < 0] = 0



In [53]:
print("RMS log-error train: ", 0.1534829529434426)
print("RMS log-error val: ", 0.15549764095917318)
print("RMS log-error train (actual): ",0.44553911117803717)
print("RMS log-error val (actual): ",0.4544472954451142)

RMS log-error train:  0.1534829529434426
RMS log-error val:  0.15549764095917318
RMS log-error train (actual):  0.4455391111780372
RMS log-error val (actual):  0.4544472954451142


In [53]:
def main_predict(model, X_test):
    X_test_mod = X_test.copy()
    output = np.array([])
    start_day, end_day = X_test['day_of_month'].min(), X_test['day_of_month'].max()
        # we lost the dates, but we still have day_of_month, which is good enough for our experiment
        
    for day in range(start_day, end_day + 1):
        pred = model.predict(X_test_mod[X_test_mod['day_of_month'] == day])
        pred[pred < 0] = 0
        print(pred)
        output = np.concatenate([output, pred], axis=0)
        for future in range(day + 1, end_day + 1):
            X_test_mod.loc[X_test_mod[X_test_mod['day_of_month'] == future].index,
                           f'sales_lag_{(future - day):02d}'] = pred
            # fill out future values now that this sales figure is available
            
    return output

In [53]:
y_pred_test = main_predict(model_rf, X_test)

[1.349292   0.02345531 1.493971   ... 2.1604116  0.56746817 2.1567235 ]

[1.3708328  0.03052545 1.2568147  ... 1.7959648  1.191319   2.2552476 ]

[1.1928338 0.        1.1708117 ... 1.6715118 0.7733267 2.1299398] 

[1.4880942 0.        1.5554705 ... 2.2369733 1.1170461 2.7174017]

[1.2262006 0.        1.2064856 ... 2.295832  1.5989244 2.6345596]

[1.5844176  0.08636409 1.481134   ... 2.2122295  1.2356842  2.0357199 ]

[1.5404915  0.27302527 1.2460707  ... 2.5592794  1.2576457  1.988879  ]

[1.407761   0.2403443  1.450003   ... 2.2944968  0.93807536 2.118688  ]

[1.3233546  0.25118655 1.2096027  ... 2.3625813  0.9701783  2.2958796 ]

[1.2763636  0.12301233 1.252281   ... 2.2097151  0.98875225 2.208721  ]

[1.502337   0.24612935 1.5411887  ... 2.7584226  1.2144794  2.8302171 ]

[1.2567537 0.1889303 1.3266869 ... 2.694969  1.474322  2.6099062]

[1.7418183  0.46445003 1.6588377  ... 2.7982423  1.0723901  2.0506146 ]

[1.4414563  0.29573742 1.4942447  ... 2.842886   1.0316128  1.9554635 ]

[1.6020162 0.5158969 1.6125051 ... 2.8743644 1.3281498 2.3716846]

[1.7183634  0.69999945 1.8148631  ... 3.0161119  1.281531   2.4809952 ]


In [53]:
delta_index = 3008016 - 3000888  # we inserted 4 Christmas days, 4 x 54 x 33 = 7128, which is the difference
submission = pd.DataFrame({'id': X_test.index - delta_index, 'sales': np.expm1(y_pred_test)})
submission.to_csv('submissionxgb.csv', index=False)
